# Recurrent Neural Network (RNN) for NLP

---

## Outline of Document
1. RNN Model with LSTM
2. Model Cross Validation
3. Evaluation Metrics

## RNN Model with LSTM



### Download Required Libraries

In [6]:
!pip install keras
!pip install tensorflow
!pip install imblearn

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.



You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


Import Required Libraries

In [7]:
import pandas as pd
import numpy as np
import csv, datetime, time, json
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from zipfile import ZipFile
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, TimeDistributed, Dense, Lambda, concatenate, Dropout, BatchNormalization, LSTM, Bidirectional
from keras.layers.merge import Concatenate
from keras.layers.embeddings import Embedding
from keras.utils.vis_utils import plot_model
from keras.regularizers import l2
from keras import backend as K
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import RandomUnderSampler


### Global Constants

In [92]:
MAX_QN_LEN = 219
MAX_WORDS = 20000
EMBED_DIM = 300 #GloVe is 300dim
MAX_SEQUENCE = 25
TRAIN_TEST = 0.1
VALIDATION = 0.1
SEED = 42
NUM_EPOCHS = 50
DROPOUT = 0.1
BATCH_SIZE = 32  #Is the default value
REGULARIZER = 0.0001
OPTIMIZER = 'adam'

MODEL_WEIGHTS_FILE_MLSTM = "question_pairs_weights_MLSTM.h5"
MODEL_WEIGHTS_FILE_MLSTM_COMB = "question_pairs_weights_MLSTM_COMB.h5"
MODEL_WEIGHTS_FILE_MLSTM_FEATURE = "question_pairs_weights_MLSTM_w_Features.h5"
MODEL_WEIGHTS_BIDIRECTIONAL_NO_HONEYCOMB = "question_pairs_weights_MLSTM_BI_NOCOMB.h5"
MODEL_WEIGHTS_BIDIRECTIONAL_HONEYCOMB = "question_pairs_weights_MLSTM_BI_COMB.h5"

### Processing Data

In [9]:
%cd "C:\Users\luciu\Desktop\NUS Lecture Notes\Y3S1\CS3244\Project"

C:\Users\luciu\Desktop\NUS Lecture Notes\Y3S1\CS3244\Project


In [10]:
train_set = pd.read_csv('features_with_word_embedding.csv', index_col=[0])

In [11]:
train_set.head(5)

,qid1,qid2,question1,question2,is_duplicate,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,unique_words_count,...,common_nouns_max,ratio_len_qn,common_word_ratio_max,common_adjectives_max,fuzz_token_set_ratio,common_words_ratio_min,common_adjectives_min,fuzz_token_sort_ratio,max_longest_substring,embed_cos_dist
id,,,,,,,,,,,,,,,,,,,,,
0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,0.478261,0.916667,100,1.000000,11,...,0.214286,1.160714,0.307692,0.0,100,1.000000,0.0,93,0.861538,0.031762
1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,0.291667,0.500000,74,0.600000,7,...,0.200000,0.574713,0.172414,0.0,86,0.700000,0.0,63,0.344828,0.266555
2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,0.166667,0.285714,46,0.172414,4,...,0.071429,1.241379,0.236111,0.0,63,0.400000,0.0,63,0.138889,0.118900
3,7,8,why am i mentally very lonely how can i solve it,find the remainder when 23 24 math is divi...,0,0.000000,0.000000,11,0.040816,0,...,0.000000,0.844828,0.224138,0.0,28,0.000000,0.0,25,0.034483,0.443512
4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.200000,0.307692,55,0.157895,4,...,0.153846,1.973684,0.213333,0.0,67,0.571429,0.0,47,0.080000,0.244168


In [12]:
Y_labels = train_set["is_duplicate"]
X_features = train_set.drop("is_duplicate", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_features, Y_labels, test_size=TRAIN_TEST, random_state=SEED)

#print(y_train_full.value_counts())

#undersample = RandomUnderSampler(sampling_strategy='majority')
# undersample = RandomUnderSampler(replacement=True)
# X_train, y_train = undersample.fit_resample(X_train_full, y_train_full)

print(y_train.value_counts())

0    229467
1    134372
Name: is_duplicate, dtype: int64


In [13]:
X_train_vector = X_train[['question1', 'question2']]
X_train_vector['question2'] = X_train_vector['question2'].astype(str)
X_train_vector.head(5)

C:\Users\luciu\AppData\Local\Temp/ipykernel_3216/2052425777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_vector['question2'] = X_train_vector['question2'].astype(str)


,question1,question2
id,,
305968,what are some examples of two kind of people,what are some cool examples of two kinds of pe...
166442,how it would be to join time for cat in februa...,how it would be to join time for cat in february
87947,what is the best way to sell my restaurant,how do i sell a restaurant idea
117082,what will be hillary clinton s foreign policy ...,what will hillary clinton s chinese policy be ...
141242,what food can increase dopamine respectively s...,what are the best supplements to increase dopa...


In [14]:
X_train_features = X_train.drop(['question1', 'question2', 'qid1', 'qid2'], axis=1)
X_train_features.head(5)

,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,unique_words_count,common_token_ratio_min,fuzz_ratio,abs_len_difference,common_stop_words_min,common_nouns_min,...,common_nouns_max,ratio_len_qn,common_word_ratio_max,common_adjectives_max,fuzz_token_set_ratio,common_words_ratio_min,common_adjectives_min,fuzz_token_sort_ratio,max_longest_substring,embed_cos_dist
id,,,,,,,,,,,,,,,,,,,,,
305968,0.411765,0.777778,86,0.477273,7,0.068182,94,6,0.444444,0.000000,...,0.000000,0.880000,0.360000,0.000000,94,0.875000,0.000000,94,0.420000,0.043271
166442,0.478261,0.916667,100,1.000000,11,0.104167,88,13,0.545455,0.181818,...,0.142857,1.270833,0.311475,0.016393,100,1.000000,0.020833,88,0.786885,0.025982
87947,0.125000,0.222222,63,0.354839,2,0.064516,55,11,0.000000,0.142857,...,0.111111,1.354839,0.309524,0.000000,65,0.285714,0.000000,52,0.261905,0.073093
117082,0.423077,0.785714,73,0.296875,11,0.062500,79,13,0.583333,0.166667,...,0.142857,1.203125,0.233766,0.000000,93,0.916667,0.000000,86,0.246753,0.074425
141242,0.235294,0.400000,69,0.351852,4,0.055556,59,10,0.142857,0.142857,...,0.100000,0.843750,0.250000,0.000000,75,0.571429,0.000000,59,0.296875,0.085473


In [15]:
X_test_vector = X_test[['question1', 'question2']]
X_test_vector['question2'] = X_test_vector['question2'].astype(str)
X_test_vector.head(5)

C:\Users\luciu\AppData\Local\Temp/ipykernel_3216/325267862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_vector['question2'] = X_test_vector['question2'].astype(str)


,question1,question2
id,,
162455,how good a phil barone saxophones,what are phil barone saxophones
34708,why have you converted to islam,why have you converted to islam what is your ...
158538,how do i learn and master things,how can i learn mastering music
61781,can dinosaurs exist on other planets,are there other worlds that have dinosaurs on ...
78367,which is the best movie from 2016,which is the best movie of 2016


In [16]:
X_test_features = X_test.drop(['question1', 'question2', 'qid1', 'qid2'], axis=1)
X_test_features.head(5)

,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,unique_words_count,common_token_ratio_min,fuzz_ratio,abs_len_difference,common_stop_words_min,common_nouns_min,...,common_nouns_max,ratio_len_qn,common_word_ratio_max,common_adjectives_max,fuzz_token_set_ratio,common_words_ratio_min,common_adjectives_min,fuzz_token_sort_ratio,max_longest_substring,embed_cos_dist
id,,,,,,,,,,,,,,,,,,,,,
162455,0.272727,0.500000,84,0.741935,3,0.096774,81,2,0.000000,0.000000,...,0.000000,1.064516,0.424242,0.000000,83,0.600000,0.000000,75,0.696970,0.072947
34708,0.375000,0.600000,100,1.000000,6,0.064516,76,20,0.666667,0.000000,...,0.000000,0.607843,0.352941,0.000000,100,1.000000,0.000000,77,0.607843,0.100991
158538,0.230769,0.428571,71,0.290323,3,0.032258,73,1,0.333333,0.000000,...,0.000000,1.032258,0.437500,0.000000,79,0.500000,0.000000,79,0.281250,0.225266
61781,0.200000,0.333333,63,0.305556,3,0.027778,42,14,0.333333,0.000000,...,0.000000,0.720000,0.260000,0.020000,67,0.500000,0.027778,58,0.220000,0.164511
78367,0.428571,0.857143,90,0.774194,6,0.096774,94,2,0.428571,0.142857,...,0.142857,1.064516,0.515152,0.030303,95,0.857143,0.032258,88,0.727273,0.000000


In [17]:
X_test_features.columns

Index(['common_words_ratio', 'common_tokens_ratio', 'fuzz_partial_ratio',
       'min_longest_substring', 'unique_words_count', 'common_token_ratio_min',
       'fuzz_ratio', 'abs_len_difference', 'common_stop_words_min',
       'common_nouns_min', 'mean_len', 'common_stop_words_max',
       'common_nouns_max', 'ratio_len_qn', 'common_word_ratio_max',
       'common_adjectives_max', 'fuzz_token_set_ratio',
       'common_words_ratio_min', 'common_adjectives_min',
       'fuzz_token_sort_ratio', 'max_longest_substring', 'embed_cos_dist'],
      dtype='object')

Tokenizer creates a keras.tokenizer object which will convert the list of strings that a user pushes in into a list of [tokens]. Based on MAX_NUMBER_OF_WORDS, the tokenizer will cut the number of tokens to that size if needed. In this case, since I put 20k as the limit, we will not be cutting anything. 

text_to_sequence fits the list of sentences into list of tokens. 
word_index provides the total number of words in this object

In [18]:
questions = X_train_vector['question1'].tolist() + X_train_vector['question2'].tolist()
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(questions)
#Training Set
question1_train = tokenizer.texts_to_sequences(X_train_vector['question1'].tolist())
question2_train = tokenizer.texts_to_sequences(X_train_vector['question2'].tolist())

#Testing Set
question1_test = tokenizer.texts_to_sequences(X_test_vector['question1'].tolist())
question2_test = tokenizer.texts_to_sequences(X_test_vector['question2'].tolist())

word_index = tokenizer.word_index

In [19]:
print("Words in index: %d" % len(word_index))

Words in index: 82370


In [20]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

glove.840b.300d.txt is a 5gb pretrained word vector file by GloVe. The format of the vectors in the txt file should be tuples of word and value. 
eg. 
(Dog, 0.12345) (Cat, 0.12000) etc. 
The code below splits the strings into tuples then forms a dictionary of dict[word] = value for every word in the pretrained model

In [21]:
embeddings_index = {}
with open('glove.840b.300d.txt', encoding='utf-8') as f:
  for line in f:
    values = line.split(' ')
    word = values[0]
    embedding = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = embedding

In [22]:
print('Word embeddings: %d' % len(embeddings_index))

Word embeddings: 2196017


In the function below, The number of words is shrunk to the min of MAX_NUMBER_OF_WORDS vs len(word_index). In this case, the number of words = MAX_NUMBER OF WORDS = 20k

For each word in word_index, we take the first 20k words from the our pretrained corpus
and try to tag each word from word_index (Which are words from our questions)
to the corpus.
This seems funny since we will expect to get gaps in the matrix when
the word does not map to any vector in the first 20k of the corpus.
Hence, when the embedding is done, we print the Null word embeddings (indicating the gap) and we see that it is only 435/20000 which can be accepted.

More tuning can be done here to increase/decrease the number of words

In [23]:
number_of_words = min(MAX_WORDS, len(word_index))
word_embedding_matrix = np.zeros((number_of_words + 1, EMBED_DIM))
for word, i in word_index.items():
  if i > MAX_WORDS:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    word_embedding_matrix[i] = embedding_vector
  
print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

Null word embeddings: 440


For LSTM, the length of our q1 and q2 has to be the same. Therefore, we pad them to an equal length of MAX_SEQUENCE_LENGTH which has been set to 25

In [24]:
q1_train_data = pad_sequences(question1_train, maxlen=MAX_SEQUENCE)
q2_train_data = pad_sequences(question2_train, maxlen=MAX_SEQUENCE)

q1_test_data =  pad_sequences(question1_test, maxlen=MAX_SEQUENCE)
q2_test_data =  pad_sequences(question2_test, maxlen=MAX_SEQUENCE)

print('Shape of question1 train data tensor:', q1_train_data.shape)
print('Shape of question2 train data tensor:', q2_train_data.shape)

print('Shape of question1 test data tensor:', q1_test_data.shape)
print('Shape of question2 test data tensor:', q1_test_data.shape)

print('Shape of label tensor:', Y_labels.shape)

Shape of question1 train data tensor: (363839, 25)
Shape of question2 train data tensor: (363839, 25)
Shape of question1 test data tensor: (40427, 25)
Shape of question2 test data tensor: (40427, 25)
Shape of label tensor: (404266,)


### Jeremy

RNN with LSTM without Dense Layers


In [25]:
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


q1_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors 
q2_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors

LSTM_DIM = 128
LSTM_DROPOUT = 0.10
LSTM_REGULARIZATION = 0.0001

q1_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q1_input)  #Creates an Embedding Object and feeds question1 into it
q2_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q2_input)

manhatten_LSTM = LSTM(LSTM_DIM, kernel_regularizer=l2(LSTM_REGULARIZATION), dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
q1_LSTM_Output = manhatten_LSTM(q1_LSTM)
q2_LSTM_Output = manhatten_LSTM(q2_LSTM)

malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([q1_LSTM_Output, q2_LSTM_Output])  #What is x[0][0]??


malstm = Model([q1_input, q2_input], [malstm_distance])
malstm.compile(loss='mean_squared_error', optimizer=OPTIMIZER, metrics=['accuracy'])
malstm.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 25, 300)      6000300     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 25, 300)      6000300     input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()

callbacks_MLSTM = [ModelCheckpoint(MODEL_WEIGHTS_FILE_MLSTM, monitor='val_accuracy', save_best_only=True)]
malstm_trained = malstm.fit([q1_train_data, q2_train_data],
                            y_train,
                            batch_size=BATCH_SIZE,
                            epochs=NUM_EPOCHS,
                            validation_split=VALIDATION,
                            verbose = 1,
                            callbacks = callbacks_MLSTM)

t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

In [27]:
malstm.save("MALSTM_UNI_NOCOMB.h5")

In [ ]:
max_val_acc, idx = max((val, idx) for (idx, val) in enumerate(malstm_trained.history['val_accuracy']))
print('Maximum validation accuracy = {0:.4f} (epoch {1:d})'.format(max_val_acc, idx+1))

In [ ]:
malstm.load_weights(MODEL_WEIGHTS_FILE_MLSTM)
loss, accuracy = malstm.evaluate([q1_test_data, q2_test_data], y_test, verbose=1)
print('Test loss = {0:.4f}, test accuracy = {1:.4f}'.format(loss, accuracy))

In [ ]:
plt.plot(malstm_trained.history['loss'], color ='blue');
plt.plot(malstm_trained.history['val_loss'], color='red');
plt.title('Training Loss Vs Validation Loss');
plt.xlabel('Epochs');
plt.ylabel('Loss');

In [ ]:
plt.plot(malstm_trained.history['accuracy'], color ='blue');
plt.plot(malstm_trained.history['val_accuracy'], color='red');
plt.title('Training Accuracy Vs Validation Accuracy');
plt.xlabel('Epochs');
plt.ylabel('Accuracy');

In [ ]:
incorrects = malstm.predict([q1_test_data, q2_test_data], verbose=1)

In [33]:
incorrects[incorrects > 0.5] = 1
incorrects[incorrects <= 0.5] = 0
flattened_incorrect = incorrects.flatten()

In [ ]:
heatmap_data = confusion_matrix(y_test, flattened_incorrect)
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0}".format(value) for value in heatmap_data.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in heatmap_data.flatten()/np.sum(heatmap_data)]
labels = [f"{group_name}\n{group_count}\n{group_percentage}" for group_name, group_count, group_percentage in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(heatmap_data, annot=labels, fmt="", cmap='Blues')

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, flattened_incorrect).ravel()
Precision = tp / (tp + fp)
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Recall = tp / (tp + fn)
F1 = 2 / ((1/Recall) + (1/Precision))

print(f"Precision: {Precision} \nAccuracy: {Accuracy} \nRecall: {Recall} \nF1: {F1}")

### Jun An
Siamese RNN with LSTM, with Dense Layers

In [36]:
# NEW MODEL TRYING TO USE SIAMNESE LSTM
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


q1_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors 
q2_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors

LSTM_DIM = 128
LSTM_DROPOUT = 0.10
LSTM_REGULARIZATION = 0.0001

q1_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q1_input)  #Creates an Embedding Object and feeds question1 into it
q2_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q2_input)

manhatten_LSTM = LSTM(LSTM_DIM, kernel_regularizer=l2(LSTM_REGULARIZATION), dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
q1_LSTM_Output = manhatten_LSTM(q1_LSTM)
q2_LSTM_Output = manhatten_LSTM(q2_LSTM)

concatenated = Concatenate()([q1_LSTM_Output, q2_LSTM_Output])
concatenated = Dropout(DROPOUT)(concatenated)
concatenated = BatchNormalization()(concatenated)

concatenated = Dense(150, kernel_regularizer=l2(REGULARIZER), activation='relu')(concatenated)
concatenated = Dropout(DROPOUT)(concatenated)
concatenated = BatchNormalization()(concatenated)

concatenated = Dense(70, kernel_regularizer=l2(REGULARIZER), activation='relu')(concatenated)
concatenated = Dropout(DROPOUT)(concatenated)
concatenated = BatchNormalization()(concatenated)

concatenated = Dense(35, kernel_regularizer=l2(REGULARIZER), activation='relu')(concatenated)
concatenated = Dropout(DROPOUT)(concatenated)
concatenated = BatchNormalization()(concatenated)

output = Dense(1, activation='sigmoid')(concatenated)

malstm_comb = Model([q1_input, q2_input], [output])
malstm_comb.compile(loss='binary_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])
malstm_comb.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 25, 300)      6000300     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 25, 300)      6000300     input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()

callbacks_MLSTM_Comb = [ModelCheckpoint(MODEL_WEIGHTS_FILE_MLSTM_COMB, monitor='val_accuracy', save_best_only=True)]
malstm_comb_trained = malstm_comb.fit([q1_train_data, q2_train_data],
                            y_train,
                            batch_size=BATCH_SIZE,
                            epochs=NUM_EPOCHS,
                            validation_split=VALIDATION,
                            verbose = 1,
                            callbacks = callbacks_MLSTM_Comb)

t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

In [ ]:
malstm_comb.save("MALSTM_UNI_COMB")

In [ ]:
max_val_acc, idx = max((val, idx) for (idx, val) in enumerate(malstm_comb_trained.history['val_accuracy']))
print('Maximum validation accuracy = {0:.4f} (epoch {1:d})'.format(max_val_acc, idx+1))

In [ ]:
malstm_comb.load_weights(MODEL_WEIGHTS_FILE_MLSTM_COMB)
loss, accuracy = malstm_comb.evaluate([q1_test_data, q2_test_data], y_test, verbose=1)
print('Test loss = {0:.4f}, test accuracy = {1:.4f}'.format(loss, accuracy))

In [ ]:
plt.plot(malstm_trained.history['loss'], color ='blue');
plt.plot(malstm_trained.history['val_loss'], color='red');
plt.title('Training Loss Vs Validation Loss');
plt.xlabel('Epochs');
plt.ylabel('Loss');

In [ ]:
plt.plot(malstm_trained.history['accuracy'], color ='blue');
plt.plot(malstm_trained.history['val_accuracy'], color='red');
plt.title('Training Accuracy Vs Validation Accuracy');
plt.xlabel('Epochs');
plt.ylabel('Accuracy');

In [ ]:
incorrects = malstm_comb.predict([q1_test_data, q2_test_data], verbose=1)

In [44]:
incorrects[incorrects > 0.5] = 1
incorrects[incorrects <= 0.5] = 0
flattened_incorrect = incorrects.flatten()

In [ ]:
heatmap_data = confusion_matrix(y_test, flattened_incorrect)
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0}".format(value) for value in heatmap_data.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in heatmap_data.flatten()/np.sum(heatmap_data)]
labels = [f"{group_name}\n{group_count}\n{group_percentage}" for group_name, group_count, group_percentage in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(heatmap_data, annot=labels, fmt="", cmap='Blues')

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, flattened_incorrect).ravel()
Precision = tp / (tp + fp)
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Recall = tp / (tp + fn)
F1 = 2 / ((1/Recall) + (1/Precision))

print(f"Precision: {Precision} \nAccuracy: {Accuracy} \nRecall: {Recall} \nF1: {F1}")

### Kay Chi

Siamese Neural Network Bidirectional LSTM, no dense layers

In [47]:
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


q1_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors 
q2_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors

LSTM_DIM = 128
LSTM_DROPOUT = 0.10
LSTM_REGULARIZATION = 0.00

q1_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q1_input)  #Creates an Embedding Object and feeds question1 into it
q2_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q2_input)

manhatten_LSTM = Bidirectional(LSTM(LSTM_DIM, kernel_regularizer=l2(LSTM_REGULARIZATION), dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT))
q1_LSTM_Output = manhatten_LSTM(q1_LSTM)
q2_LSTM_Output = manhatten_LSTM(q2_LSTM)

malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([q1_LSTM_Output, q2_LSTM_Output])  #What is x[0][0]??

malstm_bi_no_comb = Model([q1_input, q2_input], [malstm_distance])
malstm_bi_no_comb.compile(loss='mean_squared_error', optimizer=OPTIMIZER, metrics=['accuracy'])
malstm_bi_no_comb.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 25, 300)      6000300     input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 25, 300)      6000300     input_6[0][0]                    
____________________________________________________________________________________________

In [ ]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()

callbacks_MLSTM = [ModelCheckpoint(MODEL_WEIGHTS_BIDIRECTIONAL_NO_HONEYCOMB, monitor='val_accuracy', save_best_only=True)]
malstm_bi_no_comb_trained = malstm_bi_no_comb.fit([q1_train_data, q2_train_data],
                            y_train,
                            batch_size=BATCH_SIZE,
                            epochs=NUM_EPOCHS,
                            validation_split=VALIDATION,
                            verbose = 1,
                            callbacks = callbacks_MLSTM)

t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

In [ ]:
malstm_bi_no_comb.save("MALSTM_BI_NO_COMB")

In [ ]:
max_val_acc, idx = max((val, idx) for (idx, val) in enumerate(malstm_bi_no_comb_trained.history['val_accuracy']))
print('Maximum validation accuracy = {0:.4f} (epoch {1:d})'.format(max_val_acc, idx+1))

In [ ]:
malstm_bi_no_comb.load_weights(MODEL_WEIGHTS_BIDIRECTIONAL_NO_HONEYCOMB)
loss, accuracy = malstm_bi_no_comb.evaluate([q1_test_data, q2_test_data], y_test, verbose=1)
print('Test loss = {0:.4f}, test accuracy = {1:.4f}'.format(loss, accuracy))

In [ ]:
plt.plot(malstm_bi_no_comb_trained.history['loss'], color ='blue');
plt.plot(malstm_bi_no_comb_trained.history['val_loss'], color='red');
plt.title('Training Loss Vs Validation Loss');
plt.xlabel('Epochs');
plt.ylabel('Loss');

In [ ]:
plt.plot(malstm_bi_no_comb_trained.history['accuracy'], color ='blue');
plt.plot(malstm_bi_no_comb_trained.history['val_accuracy'], color='red');
plt.title('Training Accuracy Vs Validation Accuracy');
plt.xlabel('Epochs');
plt.ylabel('Accuracy');

In [ ]:
incorrects = malstm_bi_no_comb.predict([q1_test_data, q2_test_data], verbose=1)

In [56]:
incorrects[incorrects > 0.5] = 1
incorrects[incorrects <= 0.5] = 0
flattened_incorrect = incorrects.flatten()

In [ ]:
heatmap_data = confusion_matrix(y_test, flattened_incorrect)
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0}".format(value) for value in heatmap_data.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in heatmap_data.flatten()/np.sum(heatmap_data)]
labels = [f"{group_name}\n{group_count}\n{group_percentage}" for group_name, group_count, group_percentage in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(heatmap_data, annot=labels, fmt="", cmap='Blues')

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, flattened_incorrect).ravel()
Precision = tp / (tp + fp)
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Recall = tp / (tp + fn)
F1 = 2 / ((1/Recall) + (1/Precision))

print(f"Precision: {Precision} \nAccuracy: {Accuracy} \nRecall: {Recall} \nF1: {F1}")

### Penn Han
Siamese RNN with LSTM, Bidirectional with Dense Layers

In [59]:
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


q1_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors 
q2_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors

LSTM_DIM = 128
LSTM_DROPOUT = 0.10
LSTM_REGULARIZATION = 0.00

q1_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q1_input)  #Creates an Embedding Object and feeds question1 into it
q2_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q2_input)

manhatten_LSTM = Bidirectional(LSTM(LSTM_DIM, kernel_regularizer=l2(LSTM_REGULARIZATION), dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT))
q1_LSTM_Output = manhatten_LSTM(q1_LSTM)
q2_LSTM_Output = manhatten_LSTM(q2_LSTM)

concatenated = Concatenate()([q1_LSTM_Output, q2_LSTM_Output])
concatenated = Dropout(DROPOUT)(concatenated)
concatenated = BatchNormalization()(concatenated)

concatenated = Dense(150, kernel_regularizer=l2(REGULARIZER), activation='relu')(concatenated)
concatenated = Dropout(DROPOUT)(concatenated)
concatenated = BatchNormalization()(concatenated)

concatenated = Dense(70, kernel_regularizer=l2(REGULARIZER), activation='relu')(concatenated)
concatenated = Dropout(DROPOUT)(concatenated)
concatenated = BatchNormalization()(concatenated)

concatenated = Dense(35, kernel_regularizer=l2(REGULARIZER), activation='relu')(concatenated)
concatenated = Dropout(DROPOUT)(concatenated)
concatenated = BatchNormalization()(concatenated)

output = Dense(1, activation='sigmoid')(concatenated)

malstm_bi_comb = Model([q1_input, q2_input], [output])
malstm_bi_comb.compile(loss='binary_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])
malstm_bi_comb.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 25, 300)      6000300     input_7[0][0]                    
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 25, 300)      6000300     input_8[0][0]                    
____________________________________________________________________________________________

In [ ]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()

callbacks_BI_COMB_MLSTM = [ModelCheckpoint(MODEL_WEIGHTS_BIDIRECTIONAL_HONEYCOMB, monitor='val_accuracy', save_best_only=True)]
malstm_bi_comb_trained = malstm_bi_comb.fit([q1_train_data, q2_train_data],
                            y_train,
                            batch_size=BATCH_SIZE,
                            epochs=NUM_EPOCHS,
                            validation_split=VALIDATION,
                            verbose = 1,
                            callbacks = callbacks_BI_COMB_MLSTM)

t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

In [ ]:
malstm_bi_comb.save("MALSTM_BI_COMB")

In [ ]:
max_val_acc, idx = max((val, idx) for (idx, val) in enumerate(malstm_bi_comb_trained.history['val_accuracy']))
print('Maximum validation accuracy = {0:.4f} (epoch {1:d})'.format(max_val_acc, idx+1))

In [ ]:
malstm_bi_comb.load_weights(MODEL_WEIGHTS_BIDIRECTIONAL_HONEYCOMB)
loss, accuracy = malstm_bi_comb.evaluate([q1_test_data, q2_test_data], y_test, verbose=1)
print('Test loss = {0:.4f}, test accuracy = {1:.4f}'.format(loss, accuracy))

In [ ]:
plt.plot(malstm_bi_comb_trained.history['loss'], color ='blue');
plt.plot(malstm_bi_comb_trained.history['val_loss'], color='red');
plt.title('Training Loss Vs Validation Loss');
plt.xlabel('Epochs');
plt.ylabel('Loss');

In [ ]:
plt.plot(malstm_bi_comb_trained.history['accuracy'], color ='blue');
plt.plot(malstm_bi_comb_trained.history['val_accuracy'], color='red');
plt.title('Training Accuracy Vs Validation Accuracy');
plt.xlabel('Epochs');
plt.ylabel('Accuracy');

In [ ]:
incorrects = malstm_bi_comb.predict([q1_test_data, q2_test_data], verbose=1)

In [67]:
incorrects[incorrects > 0.5] = 1
incorrects[incorrects <= 0.5] = 0
flattened_incorrect = incorrects.flatten()

In [ ]:
heatmap_data = confusion_matrix(y_test, flattened_incorrect)
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0}".format(value) for value in heatmap_data.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in heatmap_data.flatten()/np.sum(heatmap_data)]
labels = [f"{group_name}\n{group_count}\n{group_percentage}" for group_name, group_count, group_percentage in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(heatmap_data, annot=labels, fmt="", cmap='Blues')

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, flattened_incorrect).ravel()
Precision = tp / (tp + fp)
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Recall = tp / (tp + fn)
F1 = 2 / ((1/Recall) + (1/Precision))

print(f"Precision: {Precision} \nAccuracy: {Accuracy} \nRecall: {Recall} \nF1: {F1}")

### Yong Shen
Siamese RNN with LSTM, with Features

In [70]:
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


q1_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors 
q2_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors

LSTM_DIM = 128
LSTM_DROPOUT = 0.10
LSTM_REGULARIZATION = 0.0001

q1_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q1_input)  #Creates an Embedding Object and feeds question1 into it
q2_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q2_input)


manhatten_LSTM = LSTM(LSTM_DIM, kernel_regularizer=l2(LSTM_REGULARIZATION), dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
q1_LSTM_Output = manhatten_LSTM(q1_LSTM)
q2_LSTM_Output = manhatten_LSTM(q2_LSTM)

malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([q1_LSTM_Output, q2_LSTM_Output])  #What is x[0][0]??

no_of_features = len(X_train_features.columns)
feature_input = Input(shape=(no_of_features,))
feature_layer = Dense(70, activation='relu')(feature_input)

concat_layer = Concatenate()([q1_LSTM_Output, q2_LSTM_Output, feature_layer])
concat_layer = Dropout(DROPOUT)(concat_layer)
concat_layer = BatchNormalization()(concat_layer)

concat_layer = Dense(150, kernel_regularizer=l2(REGULARIZER), activation='relu')(concat_layer)
concat_layer = Dropout(DROPOUT)(concat_layer)
concat_layer = BatchNormalization()(concat_layer)

concat_layer = Dense(70, kernel_regularizer=l2(REGULARIZER), activation='relu')(concat_layer)
concat_layer = Dropout(DROPOUT)(concat_layer)
concat_layer = BatchNormalization()(concat_layer)

concat_layer = Dense(35, kernel_regularizer=l2(REGULARIZER), activation='relu')(concat_layer)
concat_layer = Dropout(DROPOUT)(concat_layer)
concat_layer = BatchNormalization()(concat_layer)

output = Dense(1, activation='sigmoid')(concat_layer)

malstm_w_feature = Model([q1_input, q2_input, feature_input], [output])
malstm_w_feature.compile(loss='binary_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])
malstm_w_feature.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 25, 300)      6000300     input_9[0][0]                    
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 25, 300)      6000300     input_10[0][0]                   
____________________________________________________________________________________________

In [ ]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()

callbacks_MLSTM_feature = [ModelCheckpoint(MODEL_WEIGHTS_FILE_MLSTM_FEATURE, monitor='val_accuracy', save_best_only=True)]
malstm_w_feature_trained = malstm_w_feature.fit([q1_train_data, q2_train_data, X_train_features],
                            y_train,
                            batch_size=BATCH_SIZE,
                            epochs=NUM_EPOCHS,
                            validation_split=VALIDATION,
                            verbose = 1,
                            callbacks = callbacks_MLSTM_feature)

t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

In [ ]:
malstm_w_feature.save("MALTSM_W_FEATURES")

In [ ]:
max_val_acc, idx = max((val, idx) for (idx, val) in enumerate(malstm_w_feature_trained.history['val_accuracy']))
print('Maximum validation accuracy = {0:.4f} (epoch {1:d})'.format(max_val_acc, idx+1))

In [ ]:
malstm_w_feature.load_weights(MODEL_WEIGHTS_FILE_MLSTM_FEATURE)
loss, accuracy = malstm_w_feature.evaluate([q1_test_data, q2_test_data, X_test_features], y_test, verbose=1)
print('Test loss = {0:.4f}, test accuracy = {1:.4f}'.format(loss, accuracy))

In [ ]:
plt.plot(malstm_w_feature_trained.history['loss'], color ='blue');
plt.plot(malstm_w_feature_trained.history['val_loss'], color='red');
plt.title('Training Loss Vs Validation Loss');
plt.xlabel('Epochs');
plt.ylabel('Loss');

In [ ]:
plt.plot(malstm_w_feature_trained.history['accuracy'], color ='blue');
plt.plot(malstm_w_feature_trained.history['val_accuracy'], color='red');
plt.title('Training Accuracy Vs Validation Accuracy');
plt.xlabel('Epochs');
plt.ylabel('Accuracy');

In [ ]:
incorrects = malstm_w_feature.predict([q1_test_data, q2_test_data, X_test_features], verbose=1)

In [79]:
incorrects[incorrects > 0.5] = 1
incorrects[incorrects <= 0.5] = 0
flattened_incorrect = incorrects.flatten()

In [ ]:
heatmap_data = confusion_matrix(y_test, flattened_incorrect)
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0}".format(value) for value in heatmap_data.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in heatmap_data.flatten()/np.sum(heatmap_data)]
labels = [f"{group_name}\n{group_count}\n{group_percentage}" for group_name, group_count, group_percentage in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(heatmap_data, annot=labels, fmt="", cmap='Blues')

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, flattened_incorrect).ravel()
Precision = tp / (tp + fp)
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Recall = tp / (tp + fn)
F1 = 2 / ((1/Recall) + (1/Precision))

print(f"Precision: {Precision} \nAccuracy: {Accuracy} \nRecall: {Recall} \nF1: {F1}")